## Задача 1
 
 Есть модуль `hident.py`, который содержит логику обработки криптографических хешей. Функция `identify_hashes(input_hash)` позволяет узнать алгоритм хеширования по виду самого хеша, корутина `long_solve_hash(input_hash, alg)` имитирует работу 'обратного преобразования' хеша ([как это?](https://hashcat.net/hashcat/)), то есть позволяет восстановить секретное слово
 
 Требуется на [aiohttp](https://docs.aiohttp.org/en/stable/web.html) разработать приложение (сервер), которое будет иметь 2 ручки:
 - для вычисления возможного алгоритма хеширования по хешу `host:port/define/{hash}`
 - для вычисления секретного слова `host:port/solve?hash={hash}&algorithm={algorithm}`
 
 Напишите клиента, который будет использовать сервер:
 - отправлять 1 любой хеш, например `c4ca4238a0b923820dcc509a6f75849b`, получать его возможные алгоритмы 
 - по каждому алгоритму вычислять секретное слово
 
 Подсказки:
 - вам потребуются только GET запросы
 - смотрите атрибуты запроса `Request.match_info` и `Request.query` https://docs.aiohttp.org/en/stable/web_reference.html#request-and-base-request
 - посмотрите пример сервера с предыдущего занятия

## Задача 2 (развиваем 1ю)
 
 А теперь смоделируем таймаут сервера на долгие запросы - обернём корутину `long_solve_hash(input_hash, alg)` перед вызвовом следующим способом
 ```python
 SERVER_TIMEOUT = 4
 pwd = await asyncio.wait_for(long_solve_hash(input_hash, input_alg), SERVER_TIMEOUT)
 ```
 
Можно убедиться в том что ручка `solve` перестала работать. Как обрабатывать подобные ситуации: запросы, которые требуют очень долгого ожидания (долгой обработки) требуют реализации асинхронного api. Асинхронное api предполагает создание 2 ручек вместо одной

Текущих синхронный контракт
```go
rpc solve (SolveIn) SolveOut `Получить пароль по хешу`

message SolveIn  {inputHash string, alg string}
message SolveOut {password string}
```

Асинхронный контракт (пример)
```go
rpc createSolveTask (CreateSolveTaskIn) CreateSolveTaskOut `Создать задачу на преобразование хеша`

message CreateSolveTaskIn  {inputHash string, alg string}
message CreateSolveTaskOut {taskId int}

rpc getPassword (GetPasswordIn) GetPasswordOut `Получить результат преобразования`

message GetPasswordIn      {taskId int}
message GetPasswordOut     {password *string, err *Error}

message Error              {code int, message string}  `допустимые значения в ErrCode, ErrMessage`
const ErrCode {
    ERR_TASK_NOT_FOUND          = 1000
    ERR_TASK_NOT_FINISHED       = 1001
}
const ErrMessage {
    ERR_TASK_NOT_FOUND          = "task not found"
    ERR_TASK_NOT_FINISHED       = "task not finished"
}
```

Реализуйте в сервере предложенную схему взаимодействия.

Перепишите клиента, чтобы он мог работать с асинхронным api, поумайте над тем как должен обрабатываться статус ERR_TASK_NOT_FINISHED